In [ ]:
import asyncio
from random import random

# setting.json
SERVER = 'irc.chat.twitch.tv/alenenwooptv'# 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot_%05d' % (random() * 10000)

# util module
def parse_url(SERVER):
    temp = SERVER.split(':')
    if len(temp) != 1:
        port = int(temp[1])
    else:
        port = 6667
    return temp[0], port

class EchoClientProtocol(asyncio.Protocol):
    def __init__(self, params, loop):
        self.channel = params['channel']
        self.nick = params['nick']
        self.loop = loop

    def connection_made(self, transport):
        transport.write(('USER guest 0 * :...\r\n').encode())
        transport.write(('NICK %s\r\n' % self.nick).encode())
        transport.write(('JOIN %s\r\n' % self.channel).encode())

    def data_received(self, data):
        print(data.decode())

    def connection_lost(self, exc):
        print('The server closed the connection')
        print('Stop the event loop')
        self.loop.stop()

def main():
    host, port = parse_url(SERVER)
    print('%s %d %s %s' % (host, port, CHANNEL, NICK))
    
    params = dict()
    params['channel'] = CHANNEL
    params['nick'] = NICK
    
    loop = asyncio.get_event_loop()
    coro = loop.create_connection(lambda: EchoClientProtocol(params, loop),
                                  host, port)
    try:
        loop.run_until_complete(coro)
        loop.run_forever()
        loop.close()
    except Exception as e:
        print(e)
    
if __name__ == '__main__':
    main()